In [74]:
def read(filename):
    decks = []


    state = 'player'

    with open(filename) as f:
        for line in f:
            line = line.strip()
            if not line:
                state = 'player'
                continue

            if state == 'player':
                deck = []
                decks.append(deck)
                state = 'cards'
            elif state == 'cards':
                deck.append(int(line))    

    return decks

# Part 1

In [75]:
decks = read('input.txt')

In [76]:
debug = False

In [77]:
def printdecks(decks):
    for player, deck in enumerate(decks, 1):
        if debug: print(f"Player {player}'s deck: {', '.join(str(i) for i in deck)}")

In [78]:
round = 1
while True:
    if debug: print(f'-- Round {round} --')
    printdecks(decks)
    tops = [deck.pop(0) for deck in decks]
    for player, top in enumerate(tops, 1):
        if debug: print(f"Player {player} plays: {top}")
        
    winner = tops.index(max(tops))
    if debug: print(f'Player {winner + 1} wins the round!')
    if debug: print()

    tops.sort(reverse=True)
    
    decks[winner] += tops
    
    if any(len(deck) == 0 for deck in decks):
        break
        
    round += 1
    
if debug: print('== Post-game results ==')
if debug: printdecks(decks)

In [79]:
winning_deck = max(decks, key=len)

In [80]:
def score(winning_deck):
    return sum(position*card for position, card in enumerate(reversed(winning_deck), 1))

In [81]:
score(winning_deck)

32199

# Part 2 recursive combat

In [241]:
decks = read('input.txt')

In [242]:
def make_hashable(decks):
    return tuple(tuple(deck) for deck in decks)

In [243]:
def add_tops(tops, decks, winner):
    newcards = [tops[winner], *tops[:winner], *tops[winner+1:]]
    return [deck + (newcards if position == winner else [])
            for position, deck in enumerate(decks)]

In [244]:
debug = False

In [248]:
def recursivecombat(decks, game=1):
    seengames = set()
    
    if debug: 
        print(f'=== Game {game} ===')
        
    round = 1
    while True:
        if debug: 
            print(f'-- Round {round} (Game {game}) --')
        printdecks(decks)

        immutable_decks = make_hashable(decks)

        if immutable_decks in seengames:
            return 0, immutable_decks

        seengames.add(immutable_decks)

        tops, decks = zip(*[[top, rest] for top, *rest in decks])
        for player, top in enumerate(tops, 1):
            if debug: print(f"Player {player} plays: {top}")

                
        if any(len(deck) < top for top, deck in zip(tops, decks)):
            winner = tops.index(max(tops))
        else:
            if debug: print('Playing a sub-game to determine the winner...')
            deck_copy = [deck[:top] for deck, top in zip(decks, tops)]
            winner, winning_decks = recursivecombat(deck_copy, game + 1)
            if debug: print(f'...anyway, back to game {game}.')

        if debug: print(f'Player {winner + 1} wins round {round} of game {game}!')
        if debug: print()

        decks = add_tops(tops, decks, winner)

        if any(len(deck) == 0 for deck in decks):
            if debug: print(f'The winner of game {game} is player {winner + 1}')
            return winner, decks
            
        round += 1

In [249]:
winner, final_decks = recursivecombat(decks)
if debug: print('== Post-game results ==')
if debug: printdecks(final_decks)

In [250]:
score(final_decks[winner])

33780